In [1]:
from transformers import BloomConfig, BloomModel

# Initializing a Bloom configuration
configuration = BloomConfig()

# Initializing a model (with random weights) from the configuration
model = BloomModel(configuration)

# Accessing the model configuration
configuration = model.config

/Users/magnus.jensen@vg.no/opt/anaconda3/envs/eventyr/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
configuration

BloomConfig {
  "apply_residual_connection_post_layernorm": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_dropout": 0.0,
  "hidden_size": 64,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "bloom",
  "n_head": 8,
  "n_layer": 2,
  "pretraining_tp": 1,
  "slow_but_exact": false,
  "transformers_version": "4.26.1",
  "use_cache": true,
  "vocab_size": 250880
}

In [3]:
import torch
from transformers import AutoTokenizer, BloomForCausalLM

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
model = BloomForCausalLM.from_pretrained("bigscience/bloom-560m")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss = outputs.loss
logits = outputs.logits

In [6]:
tokenizer.decode(outputs, skip_special_tokens=True)

TypeError: argument 'ids': 'dict' object cannot be converted to 'Sequence'

In [1]:
from transformers import AutoTokenizer, TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments, AutoModelForCausalLM

model_name = "bigscience/bloom-560m" # replace with your model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/Users/magnus.jensen@vg.no/opt/anaconda3/envs/eventyr/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def generate(text):
    input_ids = tokenizer.encode(text, return_tensors='pt') 
    tensor_length = input_ids.size()[1]
    output = model.generate(
        input_ids, 
        max_length = tensor_length + 10, 
        num_beams = 2,
        no_repeat_ngram_size = 2,
        early_stopping = True,
        pad_token_id=tokenizer.eos_token_id) 
    return (tokenizer.decode(output[0], skip_special_tokens=True )) 

In [4]:
generate('Kan du norsk egentlig?')

"Kan du norsk egentlig?',\n'tooltip-pt-userpage'             =>"

In [9]:
text = 'Det var en gang en'
word = ''
while word != 'end':
    print(text)
    word = input()
    text = generate(f"{text} {word}")